In [48]:
# Please install OpenAI SDK first: `pip3 install openai`
from openai import OpenAI
from machine_lib import * 
import pandas as pd
import json

my_api_key = ""
with open("../user_config.txt", "r") as f:
    my_api_key = f.readlines()[-1].strip()
client = OpenAI(api_key=my_api_key, base_url="https://api.deepseek.com")

s = login()


b'{"user":{"id":"ZH92856"},"token":{"expiry":14400.0},"permissions":["BEFORE_AND_AFTER_PERFORMANCE_V2","TUTORIAL","WORKDAY"]}'


In [23]:
# 获取数据字段
df_datafields = get_datafields(s, dataset_id = 'pv1', region='USA', universe='TOP3000', delay=1)

def convert_datafields_to_text(df_datafields):

    #找出经过过滤和处理的数据字段
    pc_fields = process_datafields(df_datafields)
    
    clean_fields = []
    fields2pc = {}

    for pf in pc_fields:
        # feild是指数据字段，例如volume,colse,open等, pc_field是指经过过滤和处理的数据字段，例如ts_backfill(volume, 10)
        # 建立字典，存储从数据字段到经过过滤和处理的数据字段的映射关系，例如{'volume': 'ts_backfill(volume, 10)'}
        field = pf.split('ts_backfill(')[1].split(',')[0]
        clean_fields.append(field)
        fields2pc[field] = pf
    text_parts = []
    
    for index, row in df_datafields.iterrows():
        data_filed = row['id']
        if data_filed not in clean_fields:
            continue
        description = row['description']
        part = f"field: {data_filed}\n description: {description}\n field_enhanced: {fields2pc[data_filed]}\n"
        text_parts.append(part)
    return len(text_parts), ''.join(text_parts)

# 转换数据字段为文本
len_datafield, text_datafields = convert_datafields_to_text(df_datafields)

# 读取operators.xlsx文件
df_operators = pd.read_excel('operators.xlsx')
df_operators.head()

def convert_operator_to_text(df_operators):
    text_parts = []
    for index, row in df_operators.iterrows():
        operator = row['Operator']
        description = row['Description']
        example = row['Example']
        part = f"操作符: {operator}\n描述: {description}\n例子: {example}\n{'='*30}\n"
        text_parts.append(part)
    return ''.join(text_parts)

# 转换为文本
operators = convert_operator_to_text(df_operators)
# print(operators)

In [44]:
# 第一轮的系统提示词
system_prompt_round1 = """
你是一个金融量化领域专家，你首先需要完成以下工作：
1. 理解发送给你的operators的含义及使用方法；
2. 理解发送给你的金融数据的字段含义、描述、使用方法；
3. 根据operators和金融数据，构造alpha表达式。
"""

# 第一轮的用户提示词
user_prompt_round1 = """
你需要根据以下数据，构造1个alpha表达式：
operators: {operators}
financial_data: {text_datafields}
注意：
1、alpha表达式中的操作符和数据字段必须是operators和text_datafields中的内容,并保持一致。
2、构造完alpha表达式后，你需要把第一步中用到的数据字段进行增强，增强方法是：winsorize(ts_backfill(field, 120), std=4)
3、这是一个alpha表达式的例子："ts_mean(winsorize(ts_backfill(close, 120), std=4), 5) / ts_mean(winsorize(ts_backfill(volume, 120), std=4), 20)"
4、返回json格式的结果，json包含alpha_num命名的名字、alpha表示式、alpha的中文解释,json中不要出现'\n',例子：json_str={"alpha_num":名字,"alpha_expression":表达式,"alpha_explanation":解释}。
"""

system_prompt = """
你是一个金融量化领域专家，你首先需要完成以下工作：
1. 理解发送给你的operators的含义及使用方法；
2. 理解发送给你的金融数据的字段含义、描述、使用方法；注意：后续所有操作都使用增强后的数据；

在完成以上工作后，你需要完成以下工作：
1. 根据operators和金融数据，构造alpha表达式；
2. 对alpha表达式进行优化，使alpha表达式的回测结果最好；
3. 输出优化后的alpha表达式。
"""

In [45]:
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": system_prompt_round1},
        {"role": "user", "content": user_prompt_round1}
    ],
    stream=False # 不使用流式输出
)
    
    # 获取AI回复
ai_response = response.choices[0].message.content
print("AI:", ai_response)

AI: {
    "alpha_num": "alpha_001",
    "alpha_expression": "ts_mean(winsorize(ts_backfill(close, 120), std=4), 5) - ts_mean(winsorize(ts_backfill(volume, 120), std=4), 10)",
    "alpha_explanation": "5日均线价格与10日均线成交量的差值"
}


In [46]:
ai_response

'{\n    "alpha_num": "alpha_001",\n    "alpha_expression": "ts_mean(winsorize(ts_backfill(close, 120), std=4), 5) - ts_mean(winsorize(ts_backfill(volume, 120), std=4), 10)",\n    "alpha_explanation": "5日均线价格与10日均线成交量的差值"\n}'

In [47]:
import json
data = json.loads(ai_response)
data['alpha_expression']


'ts_mean(winsorize(ts_backfill(close, 120), std=4), 5) - ts_mean(winsorize(ts_backfill(volume, 120), std=4), 10)'

In [ ]:
# 记录历史
history_messages = []
round_num = 0

while True:        
    # 记录轮次，如果是第一轮，则需要发送system_prompt，并先构造alpha表达式
    if round_num == 0:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": system_prompt_round1},
                *history_messages,
                {"role": "user", "content": user_prompt_round1}
            ],
            stream=False # 不使用流式输出
        )
        alpha_expression = json.loads(response)['alpha_expression']
    
    # 获取AI回复
    ai_response = response.choices[0].message.content
    print("AI:", ai_response)
    
    # 更新历史记录（保留最近3轮）
    history_messages.extend([
        {"role": "user", "content": user_input},
        {"role": "assistant", "content": ai_response}
    ])
    history_messages = history_messages[-6:]  # 控制上下文长度
    print(history_messages)

KeyboardInterrupt: 